In [13]:
from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained("distilgpt2") 
model = AutoModelForCausalLM.from_pretrained("distilgpt2", output_hidden_states=True)

/root/anaconda3/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:774: UserWarning: `return_dict_in_generate` is NOT set to `True`, but `output_hidden_states` is. When `return_dict_in_generate` is not `True`, `output_hidden_states` is ignored.
  warnings.warn(


In [14]:
text = "The Shawshank"

# Tokenize the input string
input = tokenizer.encode(text, return_tensors="pt")

# Run the model
output = model.generate(input, max_length=5, do_sample=False)

# Print the output
print('\n',tokenizer.decode(output[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



 The Shawshank Redemption


In [15]:

# This is the embedding matrix of the model
model.transformer.wte # Dimensions are: (Number of tokens in vocabulary, dimension of model)

Embedding(50257, 768)

In [16]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# Tokenize a sample text
text = "Transformers are amazing!"
encoded_input = tokenizer(text, return_tensors="pt")

# Load a model for further processing
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased")

# Pass the tokenized input to the model
outputs = model(**encoded_input)

# Print tokenized output
print(encoded_input)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'input_ids': tensor([[  101, 19081,  2024,  6429,   999,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]])}


In [18]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import torch.nn.functional as F

# Load the tokenizer and model (e.g., GPT-2)
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2")

# Ensure the model is in evaluation mode
model.eval()

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [52]:
# Tokenize the input text
input_text = "cat sat on the"
input_ids = tokenizer.encode(input_text, return_tensors="pt")

# Get the model's output (logits)
with torch.no_grad():  # Disable gradient computation for inference
    outputs = model(input_ids)
# Extract the logits for the next token prediction
logits = outputs.logits  # Shape: [batch_size, sequence_length, vocab_size]

In [53]:
logits

tensor([[[-30.4127, -29.6745, -32.4687,  ..., -38.5662, -37.2364, -30.6165],
         [-81.5814, -81.6505, -85.2890,  ..., -90.8278, -84.9837, -84.4120],
         [-68.0099, -67.1430, -73.7267,  ..., -74.9736, -72.3875, -69.6694],
         [-91.5244, -88.8403, -92.6906,  ..., -92.1368, -92.7049, -90.3369]]])

In [54]:
# Get the logits for the last token in the sequence (next token prediction)
next_token_logits = logits[0, -1, :]  # Shape: [vocab_size]

# Apply softmax to get probabilities
probabilities = F.softmax(next_token_logits, dim=-1)

# Print the probabilities of the top 5 tokens
top_k = 5
top_k_probabilities, top_k_indices = torch.topk(probabilities, top_k)

for i in range(top_k):
    token = tokenizer.decode([top_k_indices[i].item()])
    prob = top_k_probabilities[i].item()
    print(f"Token: {token}, Probability: {prob:.4f}")


Token:  floor, Probability: 0.0845
Token:  ground, Probability: 0.0546
Token:  bench, Probability: 0.0509
Token:  couch, Probability: 0.0488
Token:  edge, Probability: 0.0422
